In [3]:
import pandas as pd


In [16]:
train_data = pd.read_csv('C:/Users/Asus/Downloads/csulb-cecs550-fall2022-ml-contest/Train.csv')
test_data = pd.read_csv('C:/Users/Asus/Downloads/csulb-cecs550-fall2022-ml-contest/Test.csv')

train_X = train_data.drop('Rhythm', axis=1)
train_Y = train_data['Rhythm']

from sklearn.preprocessing import LabelEncoder

Encoder = LabelEncoder()


train_X['Gender'] = Encoder.fit_transform(train_X['Gender'])

# train_Y = Encoder.fit_transform(train_Y)



In [17]:
# splitting the data btw train and test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.2)

In [18]:
# Standardize features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_train)
X_train = scaler.transform(X_train)

scaler.fit(X_test)
X_test = scaler.transform(X_test)

In [19]:
from sklearn.ensemble import GradientBoostingClassifier


clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.7800134138162307

In [21]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

parameters = [{"n_estimators" : [100,150,200],
               "learning_rate" : [0.1,0.09,0.01],
                "max_depth" : [1,10]
               }]
grid_search_xgboost = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 4,
                           n_jobs = -1)
grid_search_xgboost.fit(X_train, y_train)
best_accuracy_xg = grid_search_xgboost.best_score_
best_parameters = grid_search_xgboost.best_params_

In [22]:
print("best_accuracy : {:.2f} %".format(best_accuracy_xg*100))
print("Best Parameters:", best_parameters)

best_accuracy : 86.91 %
Best Parameters: {'learning_rate': 0.09, 'max_depth': 10, 'n_estimators': 200}


In [24]:
test_data['Gender'] = Encoder.fit_transform(test_data['Gender'])

results = grid_search_xgboost.predict(test_data)

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [31]:
results_df = pd.DataFrame(results, columns = ['Rhythm'])
patientID = test_data['patient_ID']
patientID_df = pd.DataFrame(patientID, columns = ['patient_ID'])
final_df = pd.concat([patientID_df, results_df], axis=1)

final_df

,patient_ID,Rhythm
0,6332,AVRT
1,7727,AVRT
2,9062,AVRT
3,6131,AVRT
4,3081,AVRT
...,...,...
3189,9522,AVRT
3190,8898,AVRT
3191,5346,AVRT
3192,4016,AVRT


In [32]:
final_df.to_csv("submission.csv")